In [40]:
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from collections import Counter
from pymystem3 import Mystem
from classification_model import train_test_datasets, NeuralNetwork, train_nn, predict, multilabel_accuracy, get_test_loader
import re
from data_preprocessing import clean_data, apply_n_grams, vectorize_text
import math
import joblib
import pickle
import numpy as np
from gensim.models import Word2Vec
import torch
import torch.nn as nn

In [41]:
stepik1 = pd.read_json('/Users/alexandrasorokovikova/Data Science/НИР Сравнение IT/Comparison-of-IT-trends/data/stepik1.json')
stepik2 = pd.read_json('/Users/alexandrasorokovikova/Data Science/НИР Сравнение IT/Comparison-of-IT-trends/data/stepik2.json')

In [42]:
stepik = pd.concat([stepik1, stepik2], ignore_index=True)
stepik = stepik[stepik['name'] != 'test']
stepik

,time,name,description,participants,price
0,20160407,Программирование на языке C++,программа курса основана на программах курсо...,"104,972",0
1,20160407,Epic Guide to Stepik,this course is about stepik.stepik is a cloud...,"4,375",0
2,20130921,Python for Programmers,based on the presentation by alex martelli.c...,"5,904",0
3,20160927,Bioinformatics Algorithms,who is this course for the course is aimed at...,"27,532",0
4,20130924,Python for Beginners,course contentloading...,"6,890",0
...,...,...,...,...,...
1808,20211004,Управление личными финансами,традиционно в россии имеющиеся проблемы приня...,3,RUB 990
1809,20211008,Технология создания убеждающей речи,"пошагово, неторопливо, выполняя не сложные за...",674,0
1810,20211013,Композитинг,курс композитинг фокусируется на работе в про...,56,0
1811,20211025,Менеджмент в сельском туризме,учебный курс подготовлен специалистами ано «а...,363,0


In [43]:
fields = {0 : 'Железо, электроника',
         1 : 'Сетевые технологии, администрирование, devops',
         2 : 'Проектирование, тестирование',
         3 : 'Информационная безопасность',
         4 : 'Маркетинг, контекстная реклама',
         5 : 'Верстка, html, javascript',
         6 : 'Машинное обучение',
         7 : 'E-commerce, финансы',
         8 : 'Дизайн интерфейсов',
         9 : 'Бизнес, CRM',
         10 : 'SEO'}

In [44]:
word2vec_model = Word2Vec.load('habr_vocab_model_on_text')

## Подготовка признаков

In [45]:
stepik_train = stepik['name']
stepik_train = pd.DataFrame(stepik_train)
stepik_train

,name
0,Программирование на языке C++
1,Epic Guide to Stepik
2,Python for Programmers
3,Bioinformatics Algorithms
4,Python for Beginners
...,...
1808,Управление личными финансами
1809,Технология создания убеждающей речи
1810,Композитинг
1811,Менеджмент в сельском туризме


In [46]:
stepik_train['name'] = clean_data(stepik_train['name'])
stepik_train.head()

100%|██████████| 1803/1803 [00:01<00:00, 1413.00it/s]


,name
0,программирование язык c
1,epic guide to stepik
2,python for programmers
3,bioinformatics algorithms
4,python for beginners


In [47]:
stepik_train['text_n_grams'] = stepik_train['name'].apply(apply_n_grams, word2vec_model=word2vec_model)
stepik_train.head()

,name,text_n_grams
0,программирование язык c,"[программирование, язык, c]"
1,epic guide to stepik,"[epic, guide, to, stepik]"
2,python for programmers,"[python, for, programmers]"
3,bioinformatics algorithms,"[bioinformatics, algorithms]"
4,python for beginners,"[python, for, beginners]"


In [48]:
stepik_train['vector'] = stepik_train['text_n_grams'].apply(vectorize_text, word2vec_model=word2vec_model)
stepik_train.head()

,name,text_n_grams,vector
0,программирование язык c,"[программирование, язык, c]","[0.7549289322413368, -2.1182537387576432, 1.54..."
1,epic guide to stepik,"[epic, guide, to, stepik]","[0.45798588110581845, -1.4590679648527778, 0.9..."
2,python for programmers,"[python, for, programmers]","[1.2364687596593233, -1.1540026915803345, 0.68..."
3,bioinformatics algorithms,"[bioinformatics, algorithms]","[-0.04114813327384081, -1.5837172714805419, 0...."
4,python for beginners,"[python, for, beginners]","[1.0016775214431637, -1.3755952052050908, 0.10..."


In [49]:
stepik_train = stepik_train.dropna(subset=['vector'])
stepik = stepik.loc[stepik_train.index]

stepik.index = np.arange(len(stepik))
stepik_train.index = np.arange(len(stepik))

In [50]:
stepik_train[list(fields.values())] = 0
stepik_train.head()

/Users/alexandrasorokovikova/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


,name,text_n_grams,vector,"Железо, электроника","Сетевые технологии, администрирование, devops","Проектирование, тестирование",Информационная безопасность,"Маркетинг, контекстная реклама","Верстка, html, javascript",Машинное обучение,"E-commerce, финансы",Дизайн интерфейсов,"Бизнес, CRM",SEO
0,программирование язык c,"[программирование, язык, c]","[0.7549289322413368, -2.1182537387576432, 1.54...",0,0,0,0,0,0,0,0,0,0,0
1,epic guide to stepik,"[epic, guide, to, stepik]","[0.45798588110581845, -1.4590679648527778, 0.9...",0,0,0,0,0,0,0,0,0,0,0
2,python for programmers,"[python, for, programmers]","[1.2364687596593233, -1.1540026915803345, 0.68...",0,0,0,0,0,0,0,0,0,0,0
3,bioinformatics algorithms,"[bioinformatics, algorithms]","[-0.04114813327384081, -1.5837172714805419, 0....",0,0,0,0,0,0,0,0,0,0,0
4,python for beginners,"[python, for, beginners]","[1.0016775214431637, -1.3755952052050908, 0.10...",0,0,0,0,0,0,0,0,0,0,0


## Поиск категорий

In [51]:
target_list = list(fields.values())
test_loader = get_test_loader(stepik_train, target_list)

In [52]:
model = NeuralNetwork()
model.load_state_dict(torch.load("nn_classifier"))
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): Tanh()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Tanh()
    (4): Linear(in_features=512, out_features=11, bias=True)
  )
)

In [53]:
predictions = predict(model, test_loader)

In [57]:
def get_field(row):
    all_fields = []
    for i in row:
        all_fields.append(fields[i])
    return all_fields

stepik['field'] = predictions
stepik['field'] = stepik['field'].apply(get_field)
stepik.head()

,time,name,description,participants,price,field
0,20160407,Программирование на языке C++,программа курса основана на программах курсо...,"104,972",0,"[Проектирование, тестирование, E-commerce, фин..."
1,20160407,Epic Guide to Stepik,this course is about stepik.stepik is a cloud...,"4,375",0,"[Проектирование, тестирование, Железо, электро..."
2,20130921,Python for Programmers,based on the presentation by alex martelli.c...,"5,904",0,"[Проектирование, тестирование]"
3,20160927,Bioinformatics Algorithms,who is this course for the course is aimed at...,"27,532",0,"[Проектирование, тестирование]"
4,20130924,Python for Beginners,course contentloading...,"6,890",0,"[Проектирование, тестирование, Машинное обучение]"


In [58]:
#stepik.to_json('stepik_final.json')